In [1]:
#from getnewspaper import getNewsPaper
#from getnewspaper import cosineSimilarity
import pandas as pd
import matplotlib as plt
import re
from bs4 import BeautifulSoup
% matplotlib inline

In [2]:
def transform_data(df_input):
    df_output = df_input.copy()
    df_output = df_output[~(df_output['description'].isna())]

    return df_output

In [3]:
df_train = pd.read_csv("train.csv", header=0,  delimiter="\t", quoting=3)

In [4]:
df_test = pd.read_csv('test.csv', header=0,  delimiter="\t", quoting=3)

In [5]:
df_other = pd.read_csv("other.csv", header=0,  delimiter="\t", quoting=3)

In [6]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170236 entries, 0 to 170235
Data columns (total 3 columns):
id             170236 non-null object
name           170179 non-null object
description    170179 non-null object
dtypes: object(3)
memory usage: 3.9+ MB


In [7]:
#df_train[df_train['description'].isna()]
df_test.shape

(170236, 3)

In [8]:
#df_train = transform_data(df_train)
df_train = df_train[~(df_train['target'].isna())]

# df_test = transform_data(df_test)
df_test = df_test[~(df_test['description'].isna())]

#df_other = transform_data(df_other)

In [9]:
df_train.head()

,id,name,description,target
0,0,Заведующий отделом/секцией в магазин YORK (Уру...,<p><strong>В НОВЫЙ МАГАЗИН YORK (хозтовары) пр...,1.0
1,1,Наладчик станков и манипуляторов с ПУ,Обязанности:работа на токарных станках с ЧПУ T...,0.0
2,2,Разработчик С++ (Криптограф),<strong>Требования:</strong> <ul> <li>Опыт про...,0.0
3,3,Фрезеровщик,<p>Условия:</p> <ul> <li>На работу вахтовым ме...,0.0
4,4,Мерчендайзер/продавец-консультант,<p><strong>Компания Палладиум Стандарт - призн...,1.0


In [10]:
def text_to_wordlist(text):
    text = re.sub('n\'t', ' not', text)
    
    text = re.sub('[^а-яА-Яa-zA-Z]', ' ', text)
    words = text.lower().split()

    #stops = set(stopwords.words("english"))

    return words

In [11]:
import nltk.data

def text_to_sentences(text):
    if pd.isnull(text):
        text = ''
        
    text = BeautifulSoup(text).get_text()
    
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(text.strip())
        
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(text_to_wordlist(raw_sentence))

    return sentences

In [12]:
from gensim.models.word2vec import Word2Vec 
def learn_W2V_model():
    %time
    sentences = []
    
    for review in df_train["description"]:
        sentences += text_to_sentences(review)
        
    %time
    for review in df_other["description"]:
        sentences += text_to_sentences(review)
    
    %time
    for review in df_test["description"]:
        sentences += text_to_sentences(review)
        
    # список параметров, которые можно менять по вашему желанию
    num_features = 300  # итоговая размерность вектора каждого слова
    min_word_count = 5  # минимальная частотность слова, чтобы оно попало в модель
    num_workers = 56     # количество ядер вашего процессора, чтоб запустить обучение в несколько потоков
    context = 10        # размер окна 
    downsampling = 1e-3 # внутренняя метрика модели
    
    model = Word2Vec(sentences, workers=num_workers, size=num_features,
                     min_count=min_word_count, window=context, sample=downsampling)
    
    return model

C:\Users\skulivec\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [13]:
# Обучение модели
model = learn_W2V_model()
model0 = model
model.init_sims(replace=True)
model.wv.save_word2vec_format('model_save_word2vec_format.vec', binary=True)

In [14]:
from gensim.models import KeyedVectors
num_features = 300
#model = KeyedVectors.load_word2vec_format('model_save_word2vec_format.md')

In [15]:
import numpy as np

def text_to_vec(words, model, size):
    text_vec = np.zeros((size,), dtype="float32")
    n_words = 0

    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            n_words = n_words + 1
            text_vec = np.add(text_vec, model[word])
    
    if n_words != 0:
        text_vec /= n_words
    return text_vec

In [16]:
def texts_to_vecs(texts, model, size):
    texts_vecs = np.zeros((len(texts), size), dtype="float32")
    
    for i, text in enumerate(texts):
        texts_vecs[i] = text_to_vec(text, model, size)

    return texts_vecs

In [17]:
# сделаем для всех текстов из train
train_like_word_list = [sum(text_to_sentences(text), []) for text in df_train['description']]

In [18]:
train_vecs = texts_to_vecs(train_like_word_list, model, num_features)

In [19]:
# сделаем тоже самое для test
test_like_word_list = [sum(text_to_sentences(text), []) for text in df_test['description']]

In [20]:
test_vecs = texts_to_vecs(test_like_word_list, model, num_features)

In [21]:
import pickle
pickle.dump(train_like_word_list, open( "train_like_word_list.dat", "wb" ) )
#train_like_word_list = pickle.load( open( "train_like_word_list.dat", "rb" ) )

pickle.dump(train_vecs, open( "train_vecs.dat", "wb" ) )
#train_vecs = pickle.load( open( "train_vecs.dat", "rb" ) )

pickle.dump(test_like_word_list, open( "test_like_word_list.dat", "wb" ) )
#test_like_word_list = pickle.load( open( "test_like_word_list.dat", "rb" ) )

pickle.dump(test_vecs, open( "test_vecs.dat", "wb" ) )
#test_vecs = pickle.load( open( "test_vecs.dat", "rb" ) )

In [22]:
test_vecs.shape, df_test.shape

((170179, 300), (170179, 3))

In [23]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
test_size=0.3
random_state=123

X = train_vecs
y = df_train["target"]
param_grid = {
    'criterion': ['gini', 'entropy'],
    #'max_depth': randint(4, 8),
    #'min_samples_leaf': randint(4, 8),
    'max_features': ['auto', 'log2', None],
    'class_weight': [None, 'balanced']}
cv = StratifiedKFold(n_splits=5, random_state=random_state, shuffle=True)

forest = RandomForestClassifier(random_state=random_state, n_estimators=150, n_jobs=-1)
random_search = RandomizedSearchCV(forest, param_distributions=param_grid, n_iter=12, n_jobs=-1,
                                   cv=cv, scoring='roc_auc', random_state=random_state) # , 'precision', 'f1', 'recall', 'roc_auc'
random_search.fit(X, y)
print(random_search.scoring)
print('random_search.best_params_')
print(random_search.best_params_)
print('random_search.best_score_')
print(random_search.best_score_)
print('random_search.best_estimator_')
print(random_search.best_estimator_)



RandomForestClassifier(n_estimators=10, criterion=’gini’, max_depth=None, 
                       min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                       max_features=’auto’, max_leaf_nodes=None, min_impurity_decrease=0.0, 
                       min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=1, 
                       random_state=None, verbose=0, warm_start=False, class_weight=None)


X, X_validate, y, y_validate = train_test_split(train_vecs, df_train["target"], test_size=test_size, random_state=random_state)

forest = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=-1,
            oob_score=False, random_state=123, verbose=0, warm_start=False)

forest = forest.fit(X, y)
predict = forest.predict(X_validate)
vals = forest.predict_proba(X_validate)[:,1]
print('Roc', roc_auc_score(y_validate, vals)) 

 Воспользуемся train_vecs, test_vecs, train["target"] 
    как матрица фичей обучающей выборки, матрица фичей тестовой выборки, таргет для обучающей выборки соответственно

 Стандартный случайный лес в таком случае


forest = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=-1,
            oob_score=False, random_state=123, verbose=0, warm_start=False)
forest = forest.fit(train_vecs, df_train["target"])
predict = forest.predict(test_vecs)


vals=forest.predict_proba(test_vecs)[:,1]
pd.DataFrame({'id':df_test['id'], 'target':[i for i in vals]}).to_csv('sampleSubmission.csv', index = False)

roc_auc
random_search.best_params_
{'max_features': 'auto', 'criterion': 'entropy', 'class_weight': None}
random_search.best_score_
0.9904878646416567
random_search.best_estimator_
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=-1,
            oob_score=False, random_state=123, verbose=0, warm_start=False)

In [24]:
from xgboost import XGBClassifier

X, X_validate, y, y_validate = train_test_split(train_vecs, df_train["target"], test_size=test_size, random_state=random_state)


model = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=648,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
#XGBClassifier(n_estimators=1000)
#XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       #colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       #max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       #n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       #reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       #silent=True, subsample=1)

model.fit(X, y)
#model.fit(X, y, 
#eval_set=[(X, y), (X_validate, y_validate)], 
#eval_metric='auc', 
#early_stopping_rounds=5)

[648]	validation_0-auc:0.993499	validation_1-auc:0.988759

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

#y_pred = model.predict_proba(X_validate)
y_pred = model.predict_proba(X_validate)[:,1]
#predictions = [round(value) for value in y_pred]

print('Roc', roc_auc_score(y_validate, y_pred)) 

y_pred

Roc 0.9571608342096216

param = {'booster' : "gbtree", 
                            objective = "binary:logistic",
                            eval_metric = 'auc', 
                            max_depth = sample(5:8, 1),
                            eta = runif(1, 0.01, 0.3),
                            gamma = runif(1, 0.15, 0.2),
                            subsample = runif(1, 0.5, 1),
                            colsample_bytree = runif(1, .5, 1),
                            min_child_weight = sample(1:10, 1),
                            num_parallel_tree = sample(1:4, 1)
                            max_delta_step = sample(1:10, 1)
        }

xgboost.cv(params, 
dtrain, 
num_boost_round=10, 
nfold=3, 
stratified=False, 
folds=None, 
metrics=(), 
obj=None, 
feval=None, 
maximize=False, 
early_stopping_rounds=None, 
fpreproc=None, 
as_pandas=True, 
verbose_eval=None, 
show_stdv=True, 
seed=0, 
callbacks=None, 
shuffle=True)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(train_vecs, df_train["target"], test_size=test_size, random_state=random_state)
#X_train, y_train = train_vecs, df_train["target"]
#X_valid, y_valid = test_vecs, df_test["target"]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier

models = [
    RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=-1,
            oob_score=False, random_state=123, verbose=0, warm_start=False),
    XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=648,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
    Pipeline([
        ('scaler', StandardScaler()),
        ('clf', LogisticRegression())
    ])    
]

In [ ]:
def roc_auc_score_cust(y_true, y_hat):
    return roc_auc_score(y_true, y_hat[:,1])

In [ ]:
from vecstack import stacking
S_train, S_valid = stacking(models,
                               X_train, y_train, X_valid,
                               regression=False,
                               mode='oof_pred_bag', 
                               needs_proba=True,
                               metric=roc_auc_score_cust,
                               n_folds=5,                
                               stratified=True,          
                               shuffle=True,             
                               random_state=123,         
                               verbose=2)         

In [ ]:
last_model = GradientBoostingClassifier(n_estimators=300, max_depth=3,
                                       learning_rate=0.01, 
                                       random_state=123)

In [ ]:
last_model.fit(S_train, y_train)
y_hat = last_model.predict_proba(S_valid)
roc_auc_score(y_valid, y_hat[:, 1])

0.9906724733360112

In [ ]:
for model in models:
    model.fit(X_train, y_train)
    y_hat = model.predict_proba(X_valid)
    score = roc_auc_score(y_valid, y_hat[:, 1])
    print('{}: {}'.format(model.__str__().split('(')[0], score))

RandomForestClassifier: 0.9902368802467469
XGBClassifier: 0.9889985218951911
Pipeline: 0.9811719514839687

In [ ]:
X_train, y_train = train_vecs, df_train["target"]
X_valid = test_vecs

In [ ]:
S_train, S_valid = stacking(models,
                               X_train, y_train, X_valid,
                               regression=False,
                               mode='oof_pred_bag', 
                               needs_proba=True,
                               metric=roc_auc_score_cust,
                               n_folds=5,                
                               stratified=True,          
                               shuffle=True,             
                               random_state=123,         
                               verbose=2)    

In [ ]:
last_model = GradientBoostingClassifier(n_estimators=300, max_depth=3,
                                       learning_rate=0.01, 
                                       random_state=123)

In [ ]:
last_model.fit(S_train, y_train)
y_hat = last_model.predict_proba(S_valid)
#roc_auc_score(y_valid, y_hat[:, 1])

task:         [classification]
n_classes:    [2]
metric:       [roc_auc_score_cust]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [RandomForestClassifier]
    fold  0:  [0.99047541]
    fold  1:  [0.98994370]
    fold  2:  [0.99085160]
    fold  3:  [0.99031660]
    fold  4:  [0.99068257]
    ----
    MEAN:     [0.99045398] + [0.00031307]
    FULL:     [0.99045241]

model  1:     [XGBClassifier]
    fold  0:  [0.98938222]
    fold  1:  [0.98815088]
    fold  2:  [0.98961011]
    fold  3:  [0.98902734]
    fold  4:  [0.98946072]
    ----
    MEAN:     [0.98912625] + [0.00052396]
    FULL:     [0.98912563]

model  2:     [Pipeline]
    fold  0:  [0.98107841]
    fold  1:  [0.97973473]
    fold  2:  [0.98188632]
    fold  3:  [0.98060899]
    fold  4:  [0.98158286]
    ----
    MEAN:     [0.98097826] + [0.00075932]
    FULL:     [0.98097408]

In [ ]:
vals=y_hat[:,1]

vals = vals[~(df_test['description'].isna())]
df_test = df_test[~(df_test['description'].isna())]

pd.DataFrame({'id':df_test['id'], 'target':[i for i in vals]}).to_csv('sampleSubmission.csv', index = False)